# Capture

In [1]:
push!(LOAD_PATH,"../src/")
using CaptureRate 
using Plots
# contains functions for:
    # solving harmonic potential (calc_harm_wave_func) 
    # general polynomial potential (calc_poly_wave_func)
    # calculating phonon overlap (calc_overlap)
    # calculating capture coefficient (calc_capt_coeff)

# input for calc_harm_wave_func: calc_harm_wave_func(ħω1, ħω2, ΔQ, ΔE; Qi=-10, Qf=10, NQ=100, nev=20, nev2=Nothing)
    # ħω1, ħω2: the energies [eV] (frequency * hbar) of each of the potentials
    # ΔQ and ΔE are the horizontal (configuration coordinate [amu^(1/2)*Å]) and vertical (energetic [eV]) differences between potential minima
    # Qi and Qf [amu^(1/2)*Å] define the domain where potential will be solved (Q ∈ [`Qi`,`Qf`]), discretised in `NQ` steps
    # `nev` the number of first energy levels of 1D potentials to solve. 
    # You can also include an additional argument nev2 = XX to solve for a different number of states in the second potential (by default nev2 = nev)

# input for calc_poly_wave_func: calc_poly_wave_func(potential_matrix_1, potential_matrix_2, poly_order, Qi=-10, Qf=10, NQ=100, nev=10, nev2=Nothing)
    # potential_matrix_1 & potential_matrix_2: potentials from files (named Potential1.txt and Potential2.txt).
    # Each potential is read from a file with Q and E in columns 1 and 2 respectively (energy in eV and Q in amu^(1/2) Angstrom). 
    # poly_order: order of the polynomial that you'd like to use to fit your potential data
    # Qi and Qf [amu^(1/2)*Å] define the domain where potential will be solved (Q ∈ [`Qi`,`Qf`]), discretised in `NQ` steps for each potential. 
    # 
    # `nev` the number of first energy levels of 1D potentials to solve. 
    # You can also include an additional argument nev2 = XX to solve for a different number of states in the second potential (by default nev2 = nev)

# input for calc_overlap: calc_overlap!(cc::CC; plt=Nothing, cut_off=0.25, σ=0.025, lplot=false)
    # cut_off: energetic difference criteria for overlap of phonons (Δϵ < cutt_off)
    # σ: amount of smearing of delta functions for determining phonon overlap

# input for calc_capt_coeff: calc_capt_coeff(W, V, T_range, cc::CC)
    # V: volume of supercell [Å³]
    # g: degeneracy (spin?)
    # W: electron-phonon coupling matrix element [ev/(amu^(1/2)*Å)]



┌ Info: Recompiling stale cache file /Users/samhood/.julia/compiled/v1.0/CaptureRate.ji for CaptureRate [top-level]
└ @ Base loading.jl:1184


# $\mathrm{GaAS}$

In [6]:
# Step 1: 

cc = CaptureRate.calc_poly_wave_func("Potential1.txt", "Potential2.txt", 5; Qi1=22, Qf1=40, Qi2=-5, Qf2=25, NQ=1000, nev=30)
CaptureRate.plot_potentials(cc)
ylims!(-330, -300)
xlims!(-2, 40)
# CaptureRate.calc_overlap!(cc; cut_off=0.25, σ=0.01)
# ylims!(-1.5, 0.5)
# xlims!(-2, 40)

In [3]:
# T_range = 1:10:800
# c = CaptureRate.calc_capt_coeff(6.4E-2, 1.1E-21, T_range, cc)
# plot(T_range, c*4*6) # degeneracy * Sommerfeld factor
# yaxis!("C (cm^-3/s)",:log10)
# xaxis!("T (K)")
# ylims!(1E-11, 2E-8)